In [1]:
import os
import cv2
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [2]:
import os
from zipfile import ZipFile


!pip install kaggle


os.environ['KAGGLE_USERNAME'] = "majisouvik1099"
os.environ['KAGGLE_KEY'] = "9464225218d56f8bfea9dd9cc437489f"


!kaggle datasets download -d jessicali9530/lfw-dataset


with ZipFile('lfw-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('lfw-dataset')


os.remove('lfw-dataset.zip')


Dataset URL: https://www.kaggle.com/datasets/jessicali9530/lfw-dataset
License(s): other




  0%|          | 0.00/112M [00:00<?, ?B/s]
  1%|          | 1.00M/112M [00:01<02:45, 708kB/s]
  2%|1         | 2.00M/112M [00:01<01:21, 1.42MB/s]
  3%|2         | 3.00M/112M [00:01<00:50, 2.25MB/s]
  4%|3         | 4.00M/112M [00:01<00:36, 3.15MB/s]
  4%|4         | 5.00M/112M [00:02<00:30, 3.70MB/s]
  5%|5         | 6.00M/112M [00:02<00:24, 4.60MB/s]
  6%|6         | 7.00M/112M [00:02<00:20, 5.42MB/s]
  7%|7         | 8.00M/112M [00:02<00:18, 5.99MB/s]
  8%|8         | 9.00M/112M [00:02<00:16, 6.70MB/s]
  9%|8         | 10.0M/112M [00:02<00:15, 7.13MB/s]
 10%|9         | 11.0M/112M [00:02<00:14, 7.59MB/s]
 11%|#         | 12.0M/112M [00:03<00:13, 7.89MB/s]
 12%|#1        | 13.0M/112M [00:03<00:13, 7.90MB/s]
 12%|#2        | 14.0M/112M [00:03<00:12, 8.01MB/s]
 13%|#3        | 15.0M/112M [00:03<00:12, 8.18MB/s]
 14%|#4        | 16.0M/112M [00:03<00:12, 8.27MB/s]
 15%|#5        | 17.0M/112M [00:03<00:12, 8.17MB/s]
 16%|#6        | 18.0M/112M [00:03<00:12, 8.18MB/s]
 17%|#6        | 19.0

In [3]:

def extract_features(image_path, model):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    image = image.unsqueeze(0)
    with torch.no_grad():
        features = model(image)
    features = features.squeeze(0)
    return features


def compute_hog(img):
    resized_img = resize(img, (128*4, 64*4))
    fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, channel_axis=-1)
    return fd


def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img, x, y):
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))
    val_ar.append(get_pixel(img, center, x, y+1))
    val_ar.append(get_pixel(img, center, x+1, y+1))
    val_ar.append(get_pixel(img, center, x+1, y))
    val_ar.append(get_pixel(img, center, x+1, y-1))
    val_ar.append(get_pixel(img, center, x, y-1))
    val_ar.append(get_pixel(img, center, x-1, y-1))
    val_ar.append(get_pixel(img, center, x-1, y))

    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val

def calcLBP(img):
    height, width, channel = img.shape
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_lbp = np.zeros((height, width,3), np.uint8)
    for i in range(0, height):
        for j in range(0, width):
             img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
    hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256])
    return hist_lbp.flatten()


In [4]:
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])
resnet.eval()

C:\Users\Veeraraju_elluru\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Veeraraju_elluru\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [5]:
lfw_folder =  r'C:\Users\Veeraraju_elluru\Desktop\Veeraraju\IITJ\Sophomore Year\Sem_2\PRML\Project\content\lfw-dataset\lfw-funneled\lfw_funneled'
X, y = [], []
for folder_name in os.listdir(lfw_folder):
    folder_path = os.path.join(lfw_folder, folder_name)
    if os.path.isdir(folder_path):
        num_images = len(os.listdir(folder_path))
        if num_images > 70:
            for image_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_name)
                image = imread(image_path)
                hog_feature = compute_hog(image)
                lbp_feature = calcLBP(image)
#                 cnn_feature = extract_features(image_path, resnet).numpy()
                hog_feature = hog_feature.reshape(-1)
                lbp_feature = lbp_feature.reshape(-1)
#                 cnn_feature = cnn_feature.flatten()
                combined_feature = np.concatenate((hog_feature, lbp_feature))

                X.append(combined_feature)
                y.append(folder_name)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [7]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [8]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
parameters = [
        {"kernel":
            ["rbf"],
            "gamma": ["auto", "scale"],
            "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000]}
]

svm_clf = SVC()
print("tuning hyperparameters via grid search")
grid = GridSearchCV(estimator=SVC(), param_grid=parameters, n_jobs=-1)
grid.fit(X_train,y_train_encoded)
print(f"grid search best score: {grid.best_score_ * 100:.2f}%")
print(f"grid search best parameters: {grid.best_params_}")



tuning hyperparameters via grid search
grid search best score: 43.20%
grid search best parameters: {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}


In [9]:
model = SVC(C= 10, degree= 2, kernel= 'poly')
model.fit(X_train,y_train_encoded)
y_pred_svm = model.predict(X_test)
# print(classification_report(y_test_encoded, y_pred_svm))
print("SVM Classifier")
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.00      0.00      0.00        16
     Colin_Powell       0.00      0.00      0.00        41
  Donald_Rumsfeld       0.00      0.00      0.00        23
    George_W_Bush       0.43      1.00      0.61       112
Gerhard_Schroeder       0.00      0.00      0.00        32
      Hugo_Chavez       0.00      0.00      0.00        12
       Tony_Blair       0.00      0.00      0.00        22

         accuracy                           0.43       258
        macro avg       0.06      0.14      0.09       258
     weighted avg       0.19      0.43      0.26       258



C:\Users\Veeraraju_elluru\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Veeraraju_elluru\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Veeraraju_elluru\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
y_pred_svm = model.predict(X_train)
accuracy_svm = accuracy_score(y_train_encoded, y_pred_svm)
print("SVM Classifier")
print("Training Accuracy:", accuracy_svm)

SVM Classifier
Training Accuracy: 0.40679611650485437


In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
model=SVC()
model.fit(X_train_pca, y_train_encoded)
y_pred = model.predict(X_test_pca)
accuracy = accuracy_score(y_test_encoded, y_pred)
print("SVM Classifier")
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_))

SVM Classifier
Accuracy: 0.43410852713178294
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.00      0.00      0.00        16
     Colin_Powell       0.00      0.00      0.00        41
  Donald_Rumsfeld       0.00      0.00      0.00        23
    George_W_Bush       0.43      1.00      0.61       112
Gerhard_Schroeder       0.00      0.00      0.00        32
      Hugo_Chavez       0.00      0.00      0.00        12
       Tony_Blair       0.00      0.00      0.00        22

         accuracy                           0.43       258
        macro avg       0.06      0.14      0.09       258
     weighted avg       0.19      0.43      0.26       258



C:\Users\Veeraraju_elluru\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Veeraraju_elluru\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Veeraraju_elluru\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
parameters = [
        {"kernel":
            ["rbf"],
            "gamma": ["auto", "scale"],
            "C": [0.1, 1, 10, 100,1000]}
]

svm_clf = SVC()
print("tuning hyperparameters via grid search")
grid = GridSearchCV(estimator=SVC(), param_grid=parameters, n_jobs=-1)
grid.fit(X_train_pca,y_train_encoded)
print(f"grid search best score: {grid.best_score_ * 100:.2f}%")
print(f"grid search best parameters: {grid.best_params_}")

tuning hyperparameters via grid search
grid search best score: 40.58%
grid search best parameters: {'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}


In [13]:
model = SVC(C= 10, gamma= 'auto', kernel= 'rbf')
model.fit(X_train_pca,y_train_encoded)
y_pred_svm = model.predict(X_test_pca)
# print(classification_report(y_test_encoded, y_pred_svm))
print("SVM Classifier")
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.00      0.00      0.00        16
     Colin_Powell       0.00      0.00      0.00        41
  Donald_Rumsfeld       0.00      0.00      0.00        23
    George_W_Bush       0.43      1.00      0.61       112
Gerhard_Schroeder       0.00      0.00      0.00        32
      Hugo_Chavez       0.00      0.00      0.00        12
       Tony_Blair       0.00      0.00      0.00        22

         accuracy                           0.43       258
        macro avg       0.06      0.14      0.09       258
     weighted avg       0.19      0.43      0.26       258



C:\Users\Veeraraju_elluru\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Veeraraju_elluru\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Veeraraju_elluru\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
y_pred_svm = model.predict(X_train_pca)
accuracy_svm = accuracy_score(y_train_encoded, y_pred_svm)
print("SVM Classifier")
print("Accuracy:", accuracy_svm)

SVM Classifier
Accuracy: 1.0


In [15]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components=None)
X_train_lda = lda.fit_transform(X_train, y_train_encoded)
X_test_lda = lda.transform(X_test)

model.fit(X_train_lda, y_train_encoded)
y_pred_svm = model.predict(X_test_lda)
accuracy_svm = accuracy_score(y_test_encoded, y_pred_svm)
print("SVM Classifier")
print("Accuracy:", accuracy_svm)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Accuracy: 0.8488372093023255
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       1.00      0.81      0.90        16
     Colin_Powell       0.93      0.95      0.94        41
  Donald_Rumsfeld       0.94      0.74      0.83        23
    George_W_Bush       0.77      0.99      0.86       112
Gerhard_Schroeder       0.94      0.50      0.65        32
      Hugo_Chavez       1.00      0.58      0.74        12
       Tony_Blair       1.00      0.73      0.84        22

         accuracy                           0.85       258
        macro avg       0.94      0.76      0.82       258
     weighted avg       0.87      0.85      0.84       258



In [16]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
parameters = [
        {"kernel":
            ["rbf"],
            "gamma": ["auto", "scale"],
            "C": [0.1, 1, 10, 100,1000]}
]

svm_clf = SVC()
print("tuning hyperparameters via grid search")
grid = GridSearchCV(estimator=SVC(), param_grid=parameters, n_jobs=-1)
grid.fit(X_train_lda,y_train_encoded)
print(f"grid search best score: {grid.best_score_ * 100:.2f}%")
print(f"grid search best parameters: {grid.best_params_}")



tuning hyperparameters via grid search
grid search best score: 82.14%
grid search best parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}


In [17]:
model = SVC(C= 0.1, gamma= 'scale', kernel= 'rbf')
model.fit(X_train_lda,y_train_encoded)
y_pred_svm = model.predict(X_test_lda)
# print(classification_report(y_test_encoded, y_pred_svm))
print("SVM Classifier")
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       1.00      0.69      0.81        16
     Colin_Powell       0.95      0.93      0.94        41
  Donald_Rumsfeld       0.93      0.61      0.74        23
    George_W_Bush       0.68      1.00      0.81       112
Gerhard_Schroeder       1.00      0.28      0.44        32
      Hugo_Chavez       1.00      0.42      0.59        12
       Tony_Blair       1.00      0.59      0.74        22

         accuracy                           0.78       258
        macro avg       0.94      0.64      0.72       258
     weighted avg       0.85      0.78      0.76       258



In [18]:
y_pred_svm = model.predict(X_train_lda)
accuracy_svm = accuracy_score(y_train_encoded, y_pred_svm)
print("SVM Classifier")
print("Accuracy:", accuracy_svm)

SVM Classifier
Accuracy: 0.8029126213592233
